In [1]:
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from random import shuffle
from string import punctuation

import matplotlib
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.max_open_warning'] = 100
matplotlib.rcParams['lines.linewidth'] = 1.0

In [3]:
from IPython.display import display

def print_full_df(df):
    pd.set_option('display.max_rows', len(df))
    display(df)
    pd.reset_option('display.max_rows')

# Grupos

## Socios activos 2016

In [4]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [5]:
%%time

dtypes = {
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Edad': 'category',
}

perfiles_df = pd.read_csv(
    'valores_grupos_2016.csv', header=0, dtype=dtypes)
# perfiles_df = perfiles_df.fillna('')

CPU times: user 66.6 ms, sys: 3.09 ms, total: 69.7 ms
Wall time: 69.2 ms


In [6]:
perfiles_df.shape

(50272, 9)

In [7]:
perfiles_df.head()

,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Edad
0,BLAA,OTRO,chapinero,bogota,CIN,universidad,OTRO,OTRO,OTRO
1,BLAA,sra.,OTRO,bogota,CEM,pensionado,OTRO,OTRO,50-64
2,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,OTRO
3,ARM,sr.,OTRO,OTRO,CEM,empleado,profesional,economía,50-64
4,BLAA,sr.,soledad,bogota,CEM,OTRO,OTRO,OTRO,OTRO


# Número de usuarios por barrio

In [8]:
usuarios_por_barrio = perfiles_df.groupby('Barrio')['Ciudad'].count()

In [9]:
usuarios_por_barrio.head()

Barrio
1 de febrero        6
1 de mayo          39
10 de mayo          4
11 de noviembre    16
12 de abril         3
Name: Ciudad, dtype: int64

# Tabla barrios

In [10]:
tabla_df = pd.read_excel('tabla_barrios.xlsx')

In [11]:
tabla_df.head()

,FID *,Shape *,OBJECTID_1,OBJECTID,Barrio,ZonaUbical,Localidad,PredNuevos,PromVenta,PromRenta,ValorM2,Shape_Leng,GlobalID,Shape_Le_1,ORIG_FID,POINT_X,POINT_Y,ESTRATO
0,1,Point,1,1,ENGATIVA ZONA URBANA,OCCIDENTE,ENGATIVA,39,3.300000e+08,0.000000e+00,1200000,3051.124910,{95D9F5AC-3E8E-4050-838E-B53229C08E36},3051.124910,0,-74.143837,4.717057,<Null>
1,2,Point,2,2,LAS FLORES,NOROCCIDENTE,SUBA,0,5.560452e+08,4.175400e+06,1150000,2718.516436,{5F47EE51-71C6-4B67-9521-E3CEBB6637C4},2718.516436,1,-74.092323,4.743632,<Null>
2,3,Point,3,3,GILMAR,NOROCCIDENTE,SUBA,0,3.465921e+08,1.662833e+06,1750000,2405.843123,{D3B5A9F7-5427-4ACE-9591-57F90F2736DC},2405.843123,2,-74.065877,4.747059,<Null>
3,4,Point,4,4,SAN PEDRO,NOROCCIDENTE,SUBA,13,1.184000e+09,2.933333e+06,800000,2620.610776,{A92B9098-0C25-49A7-89AD-1E0BF43E9EF2},2620.610776,3,-74.123515,4.748918,<Null>
4,5,Point,5,5,TIBABUYES UNIVERSAL,NOROCCIDENTE,SUBA,0,1.800000e+08,0.000000e+00,1600000,1568.529466,{29F5B269-FFD8-4D6E-AB52-9B60AF750FB3},1568.529466,4,-74.114619,4.748830,<Null>


# Calculando número de usuarios por barrio

In [12]:
localidades = [
    'Usaquén', 'Chapinero', 'Santa Fe', 'San Cristóbal', 'Usme',
    'Tunjuelito', 'Bosa', 'Kennedy', 'Fontibón', 'Engativá',
    'Suba', 'Barrios Unidos', 'Teusaquillo', 'Los Mártires', 'Antonio Nariño',
    'Puente Aranda', 'La Candelaria', 'Rafael Uribe Uribe', 'Ciudad Bolívar', 'Sumapaz'
]

In [13]:
def format_1(s):
    r = s.lower()
    r = r.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    return r

In [14]:
def format_2(s):
    r = s
    if r.startswith('el ') or r.startswith('la '):
        r = r[3:]
    elif r.startswith('los ') or r.startswith('las '):
        r = r[4:]
    return r

In [15]:
formatted_localidades_1 = [format_1(x) for x in localidades]

In [16]:
formatted_localidades_2 = list(set([format_2(x) for x in formatted_localidades_1]).difference(formatted_localidades_1))

In [17]:
%%time

def calculate_n_users(x):
    r = None
    b = x['Barrio']
    b = b.strip().lower()
    b = b.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    if b == 'juan rey (la paz)':
        b = 'juan rey'
    elif b == 'san diego-bosa':
        b = 'san diego'
    elif b.startswith('ciudad salitre'):
        b = 'ciudad salitre'
    if b in usuarios_por_barrio:
        r = usuarios_por_barrio[b]
    else: 
        if b.startswith('el ') or b.startswith('la '):
            b = b[3:]
        elif b.startswith('los ') or b.startswith('las '):
            b = b[4:]
        if b in usuarios_por_barrio:
            r = usuarios_por_barrio[b]
        else:
            for loc in formatted_localidades_1:
                if b.endswith(' ' + loc):
                    b = b[:-(len(loc) + 1)]
                    break
            if b in usuarios_por_barrio:
                r = usuarios_por_barrio[b]
            else:
                for loc in formatted_localidades_2:
                    if b.endswith(' ' + loc):
                        b = b[:-(len(loc) + 1)]
                        break
                if b in usuarios_por_barrio:
                    r = usuarios_por_barrio[b]
                else:
                    b = ' '.join(b.split()[:-1])
                    if b in usuarios_por_barrio:
                        r = usuarios_por_barrio[b]
    return r

tabla_df['nUsuarios'] = tabla_df.apply(calculate_n_users, axis=1)

CPU times: user 35.9 ms, sys: 0 ns, total: 35.9 ms
Wall time: 35.1 ms


In [18]:
tabla_df.head()

,FID *,Shape *,OBJECTID_1,OBJECTID,Barrio,ZonaUbical,Localidad,PredNuevos,PromVenta,PromRenta,ValorM2,Shape_Leng,GlobalID,Shape_Le_1,ORIG_FID,POINT_X,POINT_Y,ESTRATO,nUsuarios
0,1,Point,1,1,ENGATIVA ZONA URBANA,OCCIDENTE,ENGATIVA,39,3.300000e+08,0.000000e+00,1200000,3051.124910,{95D9F5AC-3E8E-4050-838E-B53229C08E36},3051.124910,0,-74.143837,4.717057,<Null>,NaN
1,2,Point,2,2,LAS FLORES,NOROCCIDENTE,SUBA,0,5.560452e+08,4.175400e+06,1150000,2718.516436,{5F47EE51-71C6-4B67-9521-E3CEBB6637C4},2718.516436,1,-74.092323,4.743632,<Null>,22.0
2,3,Point,3,3,GILMAR,NOROCCIDENTE,SUBA,0,3.465921e+08,1.662833e+06,1750000,2405.843123,{D3B5A9F7-5427-4ACE-9591-57F90F2736DC},2405.843123,2,-74.065877,4.747059,<Null>,18.0
3,4,Point,4,4,SAN PEDRO,NOROCCIDENTE,SUBA,13,1.184000e+09,2.933333e+06,800000,2620.610776,{A92B9098-0C25-49A7-89AD-1E0BF43E9EF2},2620.610776,3,-74.123515,4.748918,<Null>,39.0
4,5,Point,5,5,TIBABUYES UNIVERSAL,NOROCCIDENTE,SUBA,0,1.800000e+08,0.000000e+00,1600000,1568.529466,{29F5B269-FFD8-4D6E-AB52-9B60AF750FB3},1568.529466,4,-74.114619,4.748830,<Null>,5.0


In [19]:
len(tabla_df)

770

In [20]:
tabla_df['nUsuarios'].count()

642

## Resultados finales

In [21]:
tabla_df.shape

(770, 19)

In [22]:
tabla_df.to_excel('tabla_barrios_nUsuarios.xlsx', index=False)